# Docker and AWS Part One

### Steps for deploying a ML model:
1. Create and train a machine learning model (final model)
    - Save the trained model and preprocessing steps into pickle file
2. Create a flask/django/etc. API with a route for the model or models
    - Framework used will varry based on project/use
3. Dockerize the API
4. Deploy docker container to AWS

## STEP ONE: Train and MNIST model with Keras and save it as an .h5 file

In [ ]:
import keras
from keras import models
from keras import layers
from keras import backend as K
from keras.datasets import mnist

batch_size = 128
num_classes = 10
epochs = 1

# MNIST Data
mnist_data = mnist.load_data()
(x_train, y_train), (x_test, y_test) = mnist_data

# Input image dimensions
img_rows, img_cols = 28, 28

# Format data shape
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
# Normalize input shape and input type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Create sequential CNN model
model = models.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Compile and train cnn mnist model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(), metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))

In [ ]:
# Save the model a .h5 file and then delete mnist model from memory
model.save('mnist_cnn.h5')  
del model 

## STEP TWO: Create a Flask API for the saved model

In [ ]:
import werkzeug
werkzeug.cached_property = werkzeug.utils.cached_property
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from flask_restplus import Api, Resource, fields
from flask import Flask, request, jsonify
import numpy as np
from werkzeug.datastructures import FileStorage
from PIL import Image
from keras.models import model_from_json
import tensorflow as tf

app = Flask(__name__)
api = Api(app, version='1.0', title='MNIST Classification', description='CNN for MNIST')
name_space = api.namespace('Make_School', description='Methods')

single_parser = api.parser()
single_parser.add_argument('file', location='files', type=FileStorage, required=True)

model = load_model('Docker_Deployments/mnist_cnn.h5')
graph = tf.get_default_graph()

@name_space.route('/prediction')
class CNNPrediction(Resource):
    @api.doc(parser=single_parser, description='Upload an MNIST image')
    def post(self):
        # Load image file from user as a PIL image
        args = single_parser.parse_args()
        image_file = args.file
        img = Image.open(image_file)
        # Perform data preprocessing in image
        image_red = img.resize((28,28))
        image = img_to_array(image_red)
        x = image.reshape(1, 28, 28, 1)
        x = x/255
        # Make and return the prediction to the user
        with graph.as_default():
            out = model.predict(x)
        return {'prediction': str(np.argmax(out[0]))}
    
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)